**Importing & Configuring**

In [1]:
import math
import pandas as pd
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.text import OffsetFrom
import os
from scipy import stats
import pingouin as pg
from mystats import statistics, ConfidencePlotter
import sys
from plotter.HeatMapPlotter import HeatMapPlotter 
from plotter.SpacePlotter import SpacePlotter 
from plotter.SignalPlotter import SignalPlotter 
from anytree import Node, RenderTree
from utils import *

# Get the current working directory
cwd = os.getcwd()
imgDir = 'img'
pathToImgDir = os.path.join(cwd, imgDir)

c:\Python38\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.1.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


### Fetching dataframe

In [2]:
# Quantitative Dataframes
ROOT_NODE_NAME = "FACTORS"
INPUT_COLUMN_NAMES = [ 
    'Date', 'Participant', 'Task', 'Modality', 'Trial' , 
    'Target_X', 'Target_Y',
    'SC_Duration', 'SC_Count_Mean', 'SC_Count_SD', 'SC_Count_Max', 'SC_Count_Min',
    'SC_Amplitude_Mean', 'SC_Amplitude_SD', 'SC_Amplitude_Max', 'SC_Amplitude_Min',
    # eda event-related features
    'EDA_Response_Count',
    'EDA_Response_Mean',
    'EDA_Response_SD',
    'EDA_Response_Max',
    'EDA_Response_Min',

    'EDA_Level_Mean',
    'EDA_Level_SD',
    'EDA_Level_Max',
    'EDA_Level_Min',

     # ppg event-related features
    'BVP_Peak_Count',
    'BVP_Peak_Mean',
    'BVP_Peak_SD',
    'BVP_Peak_Max',
    'BVP_Peak_Min',

    'BVP_Rate_Mean',
    'BVP_Rate_SD',
    'BVP_Rate_Max',
    'BVP_Rate_Min'
    ]
parse_directory = "parse"
quant_file = 'quantitative-description.csv'
df_quant = pd.read_csv(os.path.join(parse_directory, quant_file))
column_name_diff = list(set(df_quant.columns.values.tolist()) - set(INPUT_COLUMN_NAMES))
if len(column_name_diff) != 0:
    err("Missing columns (%s)" %column_name_diff)
    exit()
df_quant['Date'] = pd.to_datetime(df_quant['Date'])
if df_quant.index.duplicated(keep='first').sum(): print("{ERROR] duplicated indexes found!")

TASK_NAMES = ['USER', 'SYSTEM']
MODALITY_NAMES = ['SMS', 'SSM']
CONTROL_NAMES = ['SMS-REST', 'SSM-REST']

df_quant = df_quant.loc[~df_quant['Modality'].isin(CONTROL_NAMES), :]

In [6]:

factor_names_list = [['All']]#, ['Task'], ['Modality'], ['Task', 'Modality']]

variable_name_list = ['SC_Duration']

new_variable_name_list =  ['Completion Time of Pin Matrix Actuation']

desc_stats_of_interest = ['count', 'mean', 'min', 'cil', 'cih', 'max']

for factor_names in factor_names_list:
    for variable_index, variable_name in enumerate(variable_name_list):
        print("%s => %s" %(' x '.join(factor_names), variable_name))
        # fetch useful information
        new_variable_name = new_variable_name_list[variable_index]
        if factor_names[0] != 'All':
            # scoping of global dataframe
            df_var = df_quant.loc[::,factor_names + [variable_name]].reset_index()
        
            # renaming columns 
            df_var_columns = {}
            for factor_index, factor_name in enumerate(factor_names):
                df_var_columns[factor_name] = "factor%s" % factor_index
            df_var_columns[variable_name] = "variable"
            df_var.rename(columns=df_var_columns, inplace=True)
            factor_columns = [factor_column for factor_column in df_var.columns.values if "factor" in factor_column]
            df_var["factor"] = df_var[factor_columns].apply(lambda x: "-".join([str(_) for _ in x]), axis=1)
            df_var.drop(columns=factor_columns, inplace=True)
            multifactor_values = df_var["factor"].unique().tolist()

            # Pivot factor x variables
            df_var_pivot = pd.DataFrame()
            for multifactor_value in multifactor_values:

                raw_values = df_var.loc[df_var['factor'] == multifactor_value, 'variable'].values
                # #remove outliers4
                # mean = np.nanmean(raw_values)
                # standard_deviation = np.nanstd(raw_values)
                # distance_from_mean = abs(raw_values - mean)
                # max_deviations = 2
                # not_outlier = distance_from_mean < max_deviations * standard_deviation
                # no_outliers = raw_values[not_outlier]
                # print(no_outliers)
                df_var_col = pd.DataFrame({multifactor_value:raw_values})
                df_var_pivot = pd.concat([df_var_pivot,df_var_col],  axis=1)
            #print(df_var)
            #df_var.plot.scatter(x='variable', y='factor', c='DarkBlue')
            #plt.scatter(x, y, s=area2, marker='o', c=c)
            #plt.show()
            # descriptive statistics
            df_stats_desc = statistics.Statistics.describePlus(df_var_pivot).round(3)
            display(df_stats_desc.loc[desc_stats_of_interest, ::])
            df_stats_infer = statistics.Statistics.inferQuantPaired(df_var_pivot).round(3)
            display(df_stats_infer)
        else :
            df_var =  df_quant.loc[::,[variable_name]].reset_index()
            display(df_var)
            



All => SC_Duration


,index,SC_Duration
0,1,3.0255
1,2,3.0505
2,3,3.0395
3,4,2.9500
4,5,3.4260
...,...,...
564,628,6.4160
565,629,6.3280
566,630,6.3350
567,631,6.2530
